In [8]:
import numpy as np
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise import SVD
from surprise.model_selection import GridSearchCV

In [9]:
def load_data():
    data_df = pd.read_csv("./data/de_data_train.csv")
    data_df['user'] = data_df['Id'].str.split('_').str[0].apply(lambda x: x[1:])
    data_df['item'] = data_df['Id'].str.split('_').str[1].apply(lambda x: x[1:])
    data_df = data_df.rename(columns={'Prediction':'rating'})
    data_df = data_df[['user','item','rating']]
    return data_df

In [10]:
data = load_data()

In [11]:
def grid_search(data, param_grid, algo, folds):
    gs = GridSearchCV(algo, 
                  param_grid, 
                  measures=['rmse', 'mae'], 
                  return_train_measures=True,
                  cv=folds, 
                  joblib_verbose=1)
    gs.fit(data)
    return gs

## SVD SGD

In [ ]:
param_grid = {'n_epochs': [5], 
              'n_factors': [10, 20], 
              'reg_all': [0.1, 0.01, 0.001], 
              'lr_all': [0.1, 0.01, 0.001]}
reader = Reader(rating_scale=(1, 5))
surprise_data = Dataset.load_from_df(data, reader=reader)
gs = grid_search(data=data, param_grid=param_grid, algo=SVD,folds=3)

In [ ]:
gs.cv_results

In [27]:
gs.best_score['rmse']

1.0138292252720091